In [412]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import plotly.graph_objects as go
from plotly.subplots import make_subplots




In [413]:
df = pd.read_csv('marketing_campaign.csv', delimiter=';')
df_original = df.copy()

In [414]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0


In [415]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

Check for duplicates 同じデーターが何回か入力しているかどうか確認

In [416]:
duplicated = df[df.duplicated()]
print(duplicated)

Empty DataFrame
Columns: [ID, Year_Birth, Education, Marital_Status, Income, Kidhome, Teenhome, Dt_Customer, Recency, MntWines, MntFruits, MntMeatProducts, MntFishProducts, MntSweetProducts, MntGoldProds, NumDealsPurchases, NumWebPurchases, NumCatalogPurchases, NumStorePurchases, NumWebVisitsMonth, AcceptedCmp3, AcceptedCmp4, AcceptedCmp5, AcceptedCmp1, AcceptedCmp2, Complain, Z_CostContact, Z_Revenue, Response]
Index: []

[0 rows x 29 columns]


Check inputted data for value errors ブーリアンの入力ミスがあるかどうか確認

In [417]:
for column in df.loc[:,'AcceptedCmp3':'Complain']:
    print(df[column].value_counts())

print(df['Response'].value_counts())

0    2077
1     163
Name: AcceptedCmp3, dtype: int64
0    2073
1     167
Name: AcceptedCmp4, dtype: int64
0    2077
1     163
Name: AcceptedCmp5, dtype: int64
0    2096
1     144
Name: AcceptedCmp1, dtype: int64
0    2210
1      30
Name: AcceptedCmp2, dtype: int64
0    2219
1      21
Name: Complain, dtype: int64
0    1906
1     334
Name: Response, dtype: int64


Check Marital_Status for unexpected inputs Marital_Statusの入力ミスがあるかどうか確認

In [418]:
print(df['Marital_Status'].value_counts())


Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
Absurd        2
YOLO          2
Name: Marital_Status, dtype: int64


Clean inputs　ミスを治す・削除

In [419]:
df['Marital_Status'].replace('Absurd',np.nan, inplace=True)
df['Marital_Status'].replace('YOLO',np.nan, inplace=True)

for status in ['Divorced','Alone','Widow']:
    df['Marital_Status'].replace(status,'Single', inplace=True)

df['Marital_Status'].replace('Married','Together', inplace=True)

print(df['Marital_Status'].value_counts())

df['Has_Child'] = df['Kidhome'] + df['Teenhome']
df['Has_Child'].where(df['Has_Child'] <= 0, 1, inplace=True)
df.drop('Kidhome',axis=1,inplace=True)
df.drop('Teenhome',axis=1,inplace=True)

print(df['Has_Child'].value_counts())


Together    1444
Single       792
Name: Marital_Status, dtype: int64
1    1602
0     638
Name: Has_Child, dtype: int64


Convert Year_Birth to Age    生年から年齢
Calculate total spending    　
Check whether any campaigns have been accepted (1-5)　いずれキャンペン１−５に参加したかどうか
Give each form of education a numeric value from lowest to highest  

In [420]:
df['Age'] = df['Year_Birth'].apply(lambda x: date.today().year - x)
df.drop('Year_Birth',axis=1,inplace=True)

df['Spending'] = df.loc[:,'MntWines':'MntGoldProds'].sum(axis=1)

df['Accepted_Cmp'] = df.loc[:,'AcceptedCmp3':'AcceptedCmp2'].sum(axis=1)
df['Accepted_Cmp'].where(df['Accepted_Cmp'] <= 0, 1, inplace=True)

education_dict = {'Basic':0,
                'Graduation':1,
                '2n Cycle':2,
                'Master':3,
                'PhD':4}
df['Education'] = df['Education'].map(education_dict)

df.dropna(axis=0,inplace=True)

In [421]:
use_col = ['ID','Age','Marital_Status','Has_Child','Education','Income','Spending','Accepted_Cmp','Response']
df = df[use_col]

In [422]:
df.describe()

,ID,Age,Has_Child,Education,Income,Spending,Accepted_Cmp,Response
count,2212.000000,2212.000000,2212.000000,2212.000000,2212.000000,2212.000000,2212.000000,2212.000000
mean,5587.731917,53.188969,0.714738,2.044756,52232.510850,606.711573,0.207052,0.149638
std,3247.944128,11.982065,0.451641,1.277182,25187.455359,603.163013,0.405285,0.356797
min,0.000000,26.000000,0.000000,0.000000,1730.000000,5.000000,0.000000,0.000000
25%,2814.750000,45.000000,0.000000,1.000000,35233.500000,68.750000,0.000000,0.000000
50%,5458.500000,52.000000,1.000000,1.000000,51381.500000,396.000000,0.000000,0.000000
75%,8421.750000,63.000000,1.000000,3.000000,68522.000000,1047.250000,0.000000,0.000000
max,11191.000000,129.000000,1.000000,4.000000,666666.000000,2525.000000,1.000000,1.000000


In [423]:
fig = make_subplots(rows=2, cols=2)
fig.add_trace(go.Box(y=df['Age'], name='Age'),row=1,col=1)
fig.add_trace(go.Box(y=df['Education'], name='Education'),row=1,col=2)
fig.add_trace(go.Box(y=df['Income'], name='Income'),row=2,col=1)
fig.add_trace(go.Box(y=df['Spending'], name='Spending'),row=2,col=2)
fig.update_layout(width=1500,height=1500)
fig.show()



Remove extreme outlier 

In [424]:
df.drop(df.index[df['Income'] == 666666],axis=0,inplace=True)

In [425]:
fig =go.Figure(go.Box(y=df['Income']))
fig.update_layout(width=600,height=600)
fig.show()

In [426]:
from sklearn.cluster import KMeans
df['Marital_Status'] = pd.get_dummies(data= df['Marital_Status'], drop_first=True)

kmeans = KMeans(n_clusters=4)
kmeans.fit(df[list(df.columns)])
df['KMeans'] = kmeans.labels_


In [427]:
df.drop('ID',axis=1).groupby('KMeans').describe()


Age                                                       \
        count       mean        std   min   25%   50%   75%    max   
KMeans                                                               
0       671.0  56.044709  11.339843  27.0  47.0  56.0  66.0  129.0   
1       675.0  53.517037  11.440373  27.0  46.0  52.0  62.0  122.0   
2       405.0  54.348148  13.380375  27.0  45.0  54.0  64.0  123.0   
3       460.0  47.539130  10.425824  26.0  39.0  46.0  53.0   76.0   

       Marital_Status            ... Accepted_Cmp      Response            \
                count      mean  ...          75%  max    count      mean   
KMeans                           ...                                        
0               671.0  0.664680  ...          0.0  1.0    671.0  0.113264   
1               675.0  0.629630  ...          0.0  1.0    675.0  0.118519   
2               405.0  0.639506  ...          1.0  1.0    405.0  0.313580   
3               460.0  0.650000  ...          0.0  1.0    460.0  0.104348   

                                           
             std  min  25%  50%  75%  max  
KMeans                                     
0       0.317151  0.0  0.0  0.0  0.0  1.0  
1       0.323461  0.0  0.0  0.0  0.0  1.0  
2       0.464522  0.0  0.0  0.0  1.0  1.0  
3       0.306044  0.0  0.0  0.0  0.0  1.0  

[4 rows x 64 columns]

In [428]:
df.drop('ID',axis=1).groupby('KMeans').mean()

,Age,Marital_Status,Has_Child,Education,Income,Spending,Accepted_Cmp,Response
KMeans,,,,,,,,
0,56.044709,0.664680,0.743666,2.169896,62817.233979,864.956781,0.198212,0.113264
1,53.517037,0.629630,0.905185,2.188148,42164.293333,223.466667,0.122963,0.118519
2,54.348148,0.639506,0.251852,2.111111,82727.624691,1421.624691,0.503704,0.313580
3,47.539130,0.650000,0.800000,1.595652,23381.952174,76.086957,0.082609,0.104348


In [429]:
from sklearn.linear_model import LinearRegression

resp_reg = LinearRegression()
x = df.loc[:,'Age':'Accepted_Cmp'].values
y = df.loc[:,'Response'].values
resp_reg.fit(x,y)


LinearRegression()

In [430]:
df['Predicted_Response'] = resp_reg.predict(x)

In [431]:
df.head()

,ID,Age,Marital_Status,Has_Child,Education,Income,Spending,Accepted_Cmp,Response,KMeans,Predicted_Response
0,5524,65,0,0,1,58138.0,1617,0,1,0,0.264735
1,2174,68,0,1,1,46344.0,27,0,0,1,0.046812
2,4141,57,1,0,1,71613.0,776,0,0,0,0.047607
3,6182,38,1,1,1,26646.0,53,0,0,3,0.022430
4,5324,41,1,1,4,58293.0,422,0,0,0,0.068923


In [432]:
df['Rounded_Prediction'] = df['Predicted_Response'].apply(lambda x: 1 if x > 0.5 else 0)
df['Prediction_Error'] = df['Rounded_Prediction']-df['Response']
df['Prediction_Error'] = df['Prediction_Error'].abs()
df.head()

,ID,Age,Marital_Status,Has_Child,Education,Income,Spending,Accepted_Cmp,Response,KMeans,Predicted_Response,Rounded_Prediction,Prediction_Error
0,5524,65,0,0,1,58138.0,1617,0,1,0,0.264735,0,1
1,2174,68,0,1,1,46344.0,27,0,0,1,0.046812,0,0
2,4141,57,1,0,1,71613.0,776,0,0,0,0.047607,0,0
3,6182,38,1,1,1,26646.0,53,0,0,3,0.022430,0,0
4,5324,41,1,1,4,58293.0,422,0,0,0,0.068923,0,0


RFM

In [433]:
df['Monetary_Rank'] = df['Spending'].rank(ascending=True) #Monetary Ranking for RFM
df['Monetary_Rank'] = (df['Monetary_Rank']/df['Monetary_Rank'].max())*100   #Normalise rank

df['Frequency'] = df_original['NumWebPurchases'] + df_original['NumCatalogPurchases'] + df_original['NumStorePurchases']
df['Frequency_Rank'] = df['Frequency'].rank(ascending=True)
df['Frequency_Rank'] = (df['Frequency_Rank']/df['Frequency_Rank'].max())*100 #Normalise rank
df.drop('Frequency', axis=1,inplace=True)

df['Recency'] = df_original['Recency']
df['Recency_Rank'] = df['Recency'].rank(ascending=False)
df['Recency_Rank'] = (df['Recency_Rank']/df['Recency_Rank'].max())*100 #Normalise rank
df.drop('Recency',axis=1,inplace=True)


In [434]:
df['Marital_Status'] = df['Marital_Status'].apply(lambda x: 'Single' if x == 0 else 'Together')

In [435]:

df.head()

,ID,Age,Marital_Status,Has_Child,Education,Income,Spending,Accepted_Cmp,Response,KMeans,Predicted_Response,Rounded_Prediction,Prediction_Error,Monetary_Rank,Frequency_Rank,Recency_Rank
0,5524,65,Single,0,1,58138.0,1617,0,1,0,0.264735,0,1,91.947523,88.280543,40.659841
1,2174,68,Single,1,1,46344.0,27,0,0,1,0.046812,0,0,7.011988,10.791855,61.251422
2,4141,57,Together,0,1,71613.0,776,0,0,0,0.047607,0,0,64.510292,80.859729,73.629124
3,6182,38,Together,1,1,26646.0,53,0,0,3,0.022430,0,0,18.706175,27.669683,73.629124
4,5324,41,Together,1,4,58293.0,422,0,0,0,0.068923,0,0,51.798236,56.153846,5.164960


In [436]:
df.to_csv('market_analysis_clean.csv')